In [1]:
# run command 'conda install scikit-image' on project's v-env
# run command 'conda install pandas' on project's v-env

In [2]:
DEBUG_TRAIN = False
DEBUG_DATA = False

In [3]:
def encode_from_utf8(x):
    return list(map(float,bin(int(x.encode().hex(),16))[2:]))

In [4]:
def decode_from_bin(x):
    res = ""
    for i in x:
        res+=str(round(i))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

In [13]:
def decode_from_bin2(x):
    res = ""
    for i in x:
        res+=str(int(i>0))
        pass
    return bytearray.fromhex(hex(int(res, 2))[2:]).decode()

In [5]:
import os
import torch
import numpy as np
import pandas as pd
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader

class KoreanHandwritingDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.dataset = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.dataset.iloc[idx, 0]
        image = io.imread(img_name)
        label = self.dataset.iloc[idx, 1]
        label = np.array([label])
        sample = {'image': image, 'label': label}

        if self.transform:
            sample = self.transform(sample)

        return sample
    pass

In [6]:
class ToTensor(object):
    def __call__(self, sample):
        image, label = sample['image'], sample['label']
        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image/255.0
        return {'image': torch.from_numpy(image.reshape(1,64,64)),
                'label': torch.tensor(encode_from_utf8(label[0]))}

In [7]:
from torchvision import transforms
csv_file = "./image-data-256/labels-map.csv"
root_dir = "./image-data-256/hangul-images"
batch_size=64
learning_rate=0.0002
num_epoch=100

korean_dataset = KoreanHandwritingDataset(csv_file,root_dir, transform=transforms.Compose([ToTensor()]))
dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = True, num_workers = 0)
#dataloader = DataLoader(korean_dataset, batch_size = batch_size, shuffle = False, num_workers = 0)

In [8]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.avgpool1 = nn.AvgPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(2,2) # kernel size 2x2 (32 = 64/2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(2,2) # kernel size 2x2 (16 = 32/2)
        self.linear1 = nn.Linear(1152,378)
        self.relu3 = nn.ReLU()
        self.linear2 = nn.Linear(378,128)
        self.relu4 = nn.ReLU()
        self.linear3 = nn.Linear(128,24)
        #self.sigmoid = nn.Sigmoid()
        pass
    def forward(self,x):
        out = self.avgpool1(x)
        if DEBUG_TRAIN : print('avgpool1: ', out.shape)
        out = self.conv1(out)
        if DEBUG_TRAIN : print('conv1: ', out.shape)
        out = self.relu1(out)
        if DEBUG_TRAIN : print('relu1: ', out.shape)
        out = self.maxpool1(out)
        if DEBUG_TRAIN : print('maxpool1: ', out.shape)
        out = self.conv2(out)
        if DEBUG_TRAIN : print('conv2: ', out.shape)
        out = self.relu2(out)
        if DEBUG_TRAIN : print('relu2: ', out.shape)
        out = self.maxpool2(out)
        if DEBUG_TRAIN : print('maxpool2: ', out.shape)
        out = out.view(out.size(0),-1)
        if DEBUG_TRAIN : print('view: ', out.shape)
        out = self.linear1(out)
        if DEBUG_TRAIN : print('linear1: ', out.shape)
        out = self.relu3(out)
        if DEBUG_TRAIN : print('relu3: ', out.shape)
        out = self.linear2(out)
        if DEBUG_TRAIN : print('linear2: ', out.shape)
        out = self.relu4(out)
        if DEBUG_TRAIN : print('relu4: ', out.shape)
        out = self.linear3(out)
        if DEBUG_TRAIN : print('linear3: ', out.shape)
        #out = self.sigmoid(out)
        #if DEBUG_TRAIN : print('sigmoid: ', out.shape)
        if DEBUG_TRAIN : print()
        return out

In [9]:
import torch.optim as optim
model=CNN().double()
loss_function=nn.BCEWithLogitsLoss()
#loss_function=nn.
optimizer= torch.optim.Adam(model.parameters(),lr=learning_rate)

> ## Training

In [10]:
import matplotlib.pyplot as plt

loss_arr=[]
for i in range(num_epoch):
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_function(output,y)
        loss.backward()
        optimizer.step()
        if j == 0:
            if DEBUG_DATA:
                plt.figure(figsize=(32, 32))
                for k, img in enumerate(x):
                    plt.subplot(8, 8, k+1)
                    plt.imshow(img.squeeze())
                    plt.xticks([])
                    plt.yticks([])
                    pass
                plt.show()
            print('epoch: ', i+1, 'sequence: ', j, loss)
            loss_arr.append(loss.detach().numpy())

epoch:  1 sequence:  0 tensor(0.6976, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  2 sequence:  0 tensor(0.6086, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  3 sequence:  0 tensor(0.4308, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  4 sequence:  0 tensor(0.4061, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  5 sequence:  0 tensor(0.3934, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  6 sequence:  0 tensor(0.3989, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  7 sequence:  0 tensor(0.3946, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  8 sequence:  0 tensor(0.3994, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  9 sequence:  0 tensor(0.3945, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  10 sequence:  0 tensor(0.3943, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  11 sequence:  0 tensor(0.3922, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  12 sequence:  0 tensor(0.3940, grad_fn=<Bina

epoch:  96 sequence:  0 tensor(0.2164, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  97 sequence:  0 tensor(0.2094, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  98 sequence:  0 tensor(0.2139, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  99 sequence:  0 tensor(0.1888, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)
epoch:  100 sequence:  0 tensor(0.2116, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


> ## Testing

In [15]:
with torch.no_grad():
    for j, sample in enumerate(dataloader):
        x = sample['image']
        y = sample['label']
        
        output = model.forward(x)
        for i in range(len(output)):
            print("출력 값: ", decode_from_bin2(output[i].tolist()),
                  ", 실제 값: ", decode_from_bin(y[i].tolist()))
            #print("실제 값: ", output[i].tolist(),
            #      ", 출력 값: ", y[i].tolist())

출력 값:  샰 , 실제 값:  쓰
출력 값:  닼 , 실제 값:  답
출력 값:  밀 , 실제 값:  버
출력 값:  뷨 , 실제 값:  불
출력 값:  혼 , 실제 값:  혼
출력 값:  쒨 , 실제 값:  체
출력 값:  뻌 , 실제 값:  들
출력 값:  샬 , 실제 값:  실
출력 값:  니 , 실제 값:  노
출력 값:  린 , 실제 값:  분
출력 값:  일 , 실제 값:  어
출력 값:  쟌 , 실제 값:  임
출력 값:  첨 , 실제 값:  물
출력 값:  샬 , 실제 값:  시
출력 값:  쇍 , 실제 값:  석
출력 값:  야 , 실제 값:  앞
출력 값:  틭 , 실제 값:  력
출력 값:  낀 , 실제 값:  늘
출력 값:  팰 , 실제 값:  편
출력 값:  삅 , 실제 값:  속
출력 값:  휨 , 실제 값:  판
출력 값:  원 , 실제 값:  월
출력 값:  띤 , 실제 값:  매
출력 값:  일 , 실제 값:  외
출력 값:  뷬 , 실제 값:  만
출력 값:  좀 , 실제 값:  절
출력 값:  뢩 , 실제 값:  복
출력 값:  엽 , 실제 값:  약
출력 값:  쎱 , 실제 값:  송
출력 값:  룼 , 실제 값:  르
출력 값:  낈 , 실제 값:  너
출력 값:  츘 , 실제 값:  처
출력 값:  읜 , 실제 값:  외
출력 값:  댬 , 실제 값:  려
출력 값:  당 , 실제 값:  당
출력 값:  잠 , 실제 값:  잘
출력 값:  홌 , 실제 값:  표
출력 값:  놔 , 실제 값:  망
출력 값:  신 , 실제 값:  신
출력 값:  삐 , 실제 값:  손
출력 값:  샽 , 실제 값:  성
출력 값:  심 , 실제 값:  십
출력 값:  솬 , 실제 값:  소
출력 값:  룼 , 실제 값:  본
출력 값:  쮀 , 실제 값:  진
출력 값:  늀 , 실제 값:  눈
출력 값:  읠 , 실제 값:  은
출력 값:  버 , 실제 값:  별
출력 값:  쎬 , 실제 값:  소
출력 값:  처 , 실제 값:  처


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte